In [ ]:
%pip install --upgrade numpy matplotlib scipy pingouin seaborn pyarrow xarray netcdf4 pyocclient

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg
import seaborn as sns
import matplotlib.pyplot as plt

# T-Tests with Pingouin

[**Pingouin**](https://pingouin-stats.org/index.html) is a statistics package in Python that wraps pandas and [scipy-stats](https://docs.scipy.org/doc/scipy/reference/stats.html), creating more-complete statistics reports and presenting the results in a readable way.


### T-Tests

T-tests compare the means of two samples of data generated from a normally-distributed population and compute the probability that they have the same mean.


| Code, | Statistical Test | Description |
| :-- | :-- | :-- |
| `pg.ttest(x, 0)` | One-Sampled T-Test | "Is the mean of my data unlikely to be zero?" |
| `pg.ttest(x, y)` | Independent-Samples T-Test | "Do the means of x and y indicate they are unlikely to be from the same distribution?" |
| `pg.ttest(x, y, paired=True)` | Paired-Samples T-test | "Comparing each value directly to its neighbor, are the mean differences of x and y's data unlikely to be zero?" |


**Exercises** let's do some analysis on some fake data to get a feel for these two statistics tools.

Generate the Data: Run the code below to create the dataset `df`.

In [ ]:
randomizer = np.random.RandomState(17)  # Makes sure the pseudorandom number generators reproduce the same data for us all.
df = pd.DataFrame()
df['A'] = randomizer.normal(0, 1, size=20)
df['B'] = randomizer.normal(0.2, 1, size=20)
df['C'] = randomizer.normal(0.7, 1, size=20)
df['D'] = df.A * 0.3 + randomizer.normal(0, 0.3, size=20)
df.plot.box()

### Analyze the Data with T-Tests in Pingouin

**A vs 0, One-Sampled T-Test (Example)**: Is the mean of the normally-distributed population that the the dataset A is generated from unlikely to be zero?

In [ ]:
pg.ttest(df['A'], 0)

**B vs 1, One-Sampled T-Test**: Is the mean of the normally-distributed population that the the dataset B is generated from unlikely to be one?

**A vs B, Independent Samples T-Test**: Is the mean of the normally-distributed population that the the dataset `A` is generated from unlikely to be the same as the mean of the normally-distributed population that the the dataset `B` is generated from?

**A vs C, Independent Samples T-Test**: Is the mean of the normally-distributed population that the the dataset `A` is generated from unlikely to be the same as the mean of the normally-distributed population that the the dataset `C` is generated from?

**A vs C, Paired Samples T-Test**: Is the mean of the differences between each pair of samples in generated from the two normally-distributed populations `A` and `C`  unlikely to be 0?

**A vs D, Paired Samples T-Test**: Is the mean of the differences between each pair of samples in generated from the two normally-distributed populations `A` and `D`  unlikely to be 0?

## T-Tests on Real-World Data: Pupil Position Analysis

In the experiments reported by [Steinmetz et al, 2019 in Nature](https://www.nature.com/articles/s41586-019-1787-x), mice performed a discrimination task where they moved the position of a stimulus using a steering wheel. During the experiment, a camera recorded the pupil position of the subject in the x and y directions.


### Download the dataset and load it into a Pandas DataFrame

In [ ]:
import owncloud

In [ ]:
client = owncloud.Client.from_public_link('https://uni-bonn.sciebo.de/s/7TrGeBWtQFGIGlY')
client.get_file('data/processed/steinmetz_2017-01-08_Muller.nc')

### Load and Filter the Data

Run the code below, which loads the file, extracts the variables, filters the data, and transforms it into a dataframe.

In [ ]:
import xarray as xr

dset = xr.load_dataset('steinmetz_2017-01-08_Muller.nc')
dd = dset[['pupil_x', 'pupil_y', 'contrast_left', 'contrast_right', 'response_type']]
dd = dd.where(
    (dset.active_trials==True) 
    & (dset.response_type != 0)
    & (dset.time >= 0.5)
    & (dset.time <= 1.3)
    , drop=True)
dd['contrast_diff'] = dd['contrast_right'] - dd['contrast_left']
trials = dd.median(dim='time')
rdf = trials.to_dataframe()
rdf.head()

**Exercises**

Extract data from the dataframe `df1`, plot its distribution(s), and use a t-test to answer the questions below:

**Example**: For the trials where `contrast_left` was greater or equal to 50 and `contrast_right` was 0, was the average `pupil_x` position for the trial zero?

In [ ]:
left = rdf[(rdf.contrast_left >= 50) & (rdf.contrast_right == 0)]
sns.kdeplot(left.pupil_x, label='left')
plt.axvline(0, color='black', linestyle='dotted')
plt.legend()
pg.ttest(left.pupil_x, 0)

Were the mice looking at the stimulus that was presented?  In trials where there was only one stimulus present (i.e. `contrast_left` or `contrast_right` was zero), and the stimulus had at least a contrast level of 50 (quite visible), was there a difference in the average `pupil_x` position?

As a sanity check, was there a difference in vertical eye position between the left-stimulus trials and the right-stimulus trials?  In trials where there was only one stimulus present (i.e. `contrast_left` or `contrast_right` was zero), and the stimulus had at least a contrast level of 50 (quite visible), was there a difference in the average `pupil_y` position?

Did mice look in the direction they turned the wheel?  Was there a difference in average `pupil_x` position between left-turning trials (i.e. `response_type` is -1) and right-turning trials (i.e. `response_type` is 1)?

As a sanity check, did vertical eye position matter?  Was there a difference in average `pupil_y` position between left-turning trials (i.e. `response_type` is -1) and right-turning trials (i.e. `response_type` is 1)?

## Working with Long DataFrames

If data is organized as a long dataframe, it is also possible to have `pingouin` simply do all the comparisons between the groups.  Not only is this very convenient; it also makes it possible for pingouin to do richer analyses.

| Test, | Function | 
| :--- | :--- |
| Melt | `pd.melt()` |
| ANOVA | `pg.anova(data=dfl, dv='value', between=['variable'], detailed=True, )` |
| ANCOVA | `pg.ancova(dfl, dv='value', between=['variable'], covar=['index'], )` |
| Pairwise T-tests | `pg.pairwise_tests(data=dfl, dv='value', between='variable', subject='index', padjust='fdr')`  |  

**Exercises**

Use the `dfl` long dataframe below:

In [ ]:
dfl = df.melt(ignore_index=False).reset_index()
dfl.sample(5)

**Example**: Before doing pairwise tests, it's often helpful to run an ANOVA analysis, which can tell you quickly whether a pairwise difference exists within a collection of potential pairs.  Below, run an ANOVA test comparing the dv 'value' between values of 'variable' to see if a significant difference is expected.

In [ ]:
pg.anova(dfl, dv='value', between=['variable'], detailed=True, )

Do a t-tests for every possible pairwise comparison of "variable" (e.g. "A" vs "B", "A" vs "C", etc).  (Tip: it works similar to the `pg.anova` function)

Repeat the tests, but his time, adjust the p-value to compensate for multiple comparisons using the 'fdr' method.  Are the significant differences still significant?  How do the corrected p-values compare to the uncorrected p-values?

Do a pairwise test again on the long dataframe, comparing the value of 'value' between each value 'variable'.  This time, use a non-parametric comparison (not assuming a normal distribution of each variable), and still adjust for multiple comparisons.  Are your results the same?

Use the `rdf` dataframe from the pupil measurement data.  Is there a significant different in average horizontal position, when comparing all the `contrast_diff` levels in the study pairwise?

## A Gallery of Plots : 2D Histograms with `sns.jointplot()`

Let's try out a few different plots, for comparing two different continuous variables' distributions.

| Code | Description |
| :--  | :-- |
| `sns.jointplot(df, x='colA', y='colB')` | Makes a joint histogram |

**Exercises**

**Example**: Make a joint plot comparing the `pupil_x` and `pupil_y` variables:

In [ ]:
sns.jointplot(rdf, x="pupil_x", y="pupil_y");

Make a joint plot comparing the `pupil_x` and `pupil_y` variables, using `kind='kde'`.

Make a joint plot comparing the `pupil_x` and `pupil_y` variables, using `kind='kde'` and setting `hue` to `'response_type'`.

Make a joint plot comparing the `pupil_x` and `pupil_y` variables, using `kind='hex'`.

## Further Reading

Nice article on Pingouin here: https://towardsdatascience.com/the-new-kid-on-the-statistics-in-python-block-pingouin-6b353a1db57c

Nice summary of the different effect size metrics and when to pick which: https://www.socscistatistics.com/effectsize/default3.aspx

## Extra: Correlations and Simpson's Paradox

| Function | Description |
| :-- | :-- |
| `pg.corr(A, B)` | The correlation between variables A and B |
| `pg.pairwise_corr(flowers)` | Pairwise correlations |
| `df.plot.scatter(x='A', y='B')` | Make a scatter plot |
| `sns.lmplot(data=df, x='Col1', y='Col2', hue='Col3')` | Make a scatter plot with a regression line |

Download the iris dataset for this section:

In [ ]:
%pip install bokeh bokeh_sampledata

In [ ]:
from bokeh import sampledata
# sampledata.download()

In [ ]:
from bokeh.sampledata import iris
flowers = iris.flowers
flowers.head()

**Exercises**:

Is there a correlation between flowers' sepal widths and their petal lengths?

Use seaborn's `lmplot()` function to plot a regression line showing the sepal widths and petal lengths.  Does the direction of this line match the results?  Looking closely at the data's distribution, what is missing from this story?

This time factor in the `species` variable into the `lmplot()` visualization.  What kind of relationship do you see between `sepal_width` and `petal_length` now?

What's happening here is that a third variable, `"species"`, is influencing both the petal and the sepal shape; this effect is called "Simpson's Paradox".   Unfortunately, at this time (as far as I know) pengouin doesn't have a way to split a correlation into multiple groups; one way to do this, though, is to use a `df.groupby()` analysis:

In [ ]:
fun = lambda data: pg.corr(data['sepal_width'], data['petal_length'])
flowers.groupby('species').apply(fun, include_groups=False)

To summarize, accounting for the right variables is a big part of science; we should always be aware that the patterns we see in our data can be turned completely upside down when new aspects of a problem are understood and factored in!  